In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import urllib.request
import sys

# ==========================================================
# Step 1: Download Protein Structure (Villin Headpiece)
# ==========================================================
# This solves the FileNotFoundError by fetching the structure directly from RCSB PDB.
pdb_id = '1l2y'
pdb_filename = 'Trp-cage.pdb'

print(f"Downloading protein structure ({pdb_id})...")
urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb_id}.pdb', pdb_filename)
pdb = PDBFile(pdb_filename)

# Using Amber14 force field and TIP3P water model
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# ==========================================================
# Step 2: Set up the Simulation System
# ==========================================================
print("Creating the simulation system...")
# NonbondedMethod=NoCutoff is used for vacuum simulation (faster for testing)
system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff, constraints=HBonds)

# Integrator settings: 300K temperature, 2fs time step
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# ==========================================================
# Step 3: Energy Minimization
# ==========================================================
print("Performing energy minimization...")
simulation.minimizeEnergy()

# ==========================================================
# Step 4: Run 100 ps Production Simulation
# ==========================================================
# 100 ps / 0.002 ps = 50,000 steps
total_steps = 50000 
record_interval = 1000 # Save a frame every 1000 steps

# Reporters: Output trajectory (PDB) and simulation data (CSV-like format to console)
simulation.reporters.append(PDBReporter('output_100ps.pdb', record_interval))
simulation.reporters.append(StateDataReporter(sys.stdout, record_interval, step=True, 
                             potentialEnergy=True, temperature=True, speed=True, progress=True, totalSteps=total_steps))

print(f"Starting 100 ps simulation ({total_steps} steps)...")
simulation.step(total_steps)

print("-" * 30)
print("✅ Task Completed Successfully!")
print(f"Files generated: {pdb_filename} (initial), output_100ps.pdb (trajectory)")

Creating the simulation system...
Performing energy minimization...
Starting 100 ps simulation (50000 steps)...
#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
2.0%,1000,-1297.1138801574707,263.1424137311148,0
4.0%,2000,-1221.5121726989746,266.8870942543487,1.42e+03
6.0%,3000,-1295.359088897705,295.89852732559444,1.45e+03
8.0%,4000,-1273.2529258728027,299.9745419260873,1.46e+03
10.0%,5000,-1311.0704307556152,292.9442398043383,1.46e+03
12.0%,6000,-1213.373764038086,293.13895104385534,1.43e+03
14.0%,7000,-1308.9733810424805,307.4221211051444,1.42e+03
16.0%,8000,-1262.1323928833008,313.78354582263546,1.42e+03
18.0%,9000,-1234.5786209106445,313.84811428255085,1.42e+03
20.0%,10000,-1091.8167190551758,314.77474325312767,1.43e+03
22.0%,11000,-1193.7829055786133,329.1297742576719,1.43e+03
24.0%,12000,-1218.2294731140137,298.98972783644956,1.43e+03
26.0%,13000,-1232.247402191162,294.5619911548779,1.43e+03
28.0%,14000,-1198.2006759643555,318.59594649278614,